# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
city_csv = "city_df.csv"
city_df = pd.read_csv("city_df.csv")

city_df.head()




,Unnamed: 0,CityID,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
0,0,3833367,Ushuaia,65,46.40,-54.80,40,16.11,46.40,-68.30,"{'lon': -68.3, 'lat': -54.8}",AR
1,1,3471451,Arraial do Cabo,85,74.59,-22.97,100,3.15,74.59,-42.03,"{'lon': -42.03, 'lat': -22.97}",BR
2,2,5122534,Jamestown,85,16.32,42.10,20,5.82,21.99,-79.24,"{'lon': -79.24, 'lat': 42.1}",US
3,3,2163355,Hobart,25,83.16,-42.88,100,24.16,84.99,147.33,"{'lon': 147.33, 'lat': -42.88}",AU
4,4,1507390,Dikson,86,-12.28,73.51,100,11.36,-12.28,80.55,"{'lon': 80.55, 'lat': 73.51}",RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:
locations = city_df[["Latitude","Longitude"]]
humidity = city_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=90, 
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
maxtemp_df = city_df[(city_df['MaxTemp'] < 90) & (city_df['MaxTemp'] > 73)]


windspeed_df = maxtemp_df[(maxtemp_df['WindSpeed'] < 10)]
perfect_df = windspeed_df[(windspeed_df['Cloudiness'] == 0)].dropna()
perfect_df

,Unnamed: 0,CityID,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
64,64,3985168,San Patricio,73,81.45,19.22,0,7.11,81.45,-104.70,"{'lon': -104.7, 'lat': 19.22}",MX
188,188,1831173,Kampong Cham,56,83.39,11.99,0,3.38,83.39,105.46,"{'lon': 105.46, 'lat': 11.99}",KH
261,261,3836277,Santa Fe,85,74.48,-31.63,0,8.01,75.99,-60.70,"{'lon': -60.7, 'lat': -31.63}",AR
280,280,2294034,Winneba,88,80.60,5.35,0,4.70,80.60,-0.62,"{'lon': -0.62, 'lat': 5.35}",GH
332,332,1649150,Bengkulu,57,89.33,-3.80,0,0.38,89.33,102.27,"{'lon': 102.27, 'lat': -3.8}",ID
496,496,1258534,Ranavav,35,73.40,21.68,0,9.17,73.40,69.74,"{'lon': 69.74, 'lat': 21.68}",IN
513,513,1830377,Lumphat,44,82.22,13.49,0,5.66,82.22,106.98,"{'lon': 106.98, 'lat': 13.49}",KH
519,519,2094144,Kerema,69,86.70,-7.96,0,7.70,86.70,145.78,"{'lon': 145.78, 'lat': -7.96}",PG


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = pd.DataFrame(columns= ['hotel_name','CityID']) 


for index,row in perfect_df.iterrows():
    target_coordinates = f"{row['Latitude']},{row['Longitude']}"
    target_search = "Hotels"
    target_radius = 5000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    response = response.json()
    
    if response['status'] == 'OK':
        hotel_df = hotel_df.append({'hotel_name' : response["results"][0]["name"],
                                'CityID' : row['CityID']}, ignore_index=True)
    else:
        print('No hotel located')
        
hotel_df

No hotel located


,hotel_name,CityID
0,Grand Isla Navidad Resort,3985168
1,LBN Asian Hotel,1831173
2,InterTower Hotel,3836277
3,Lagoon Lodge,2294034
4,Hotel Santika Bengkulu,1649150
5,Lumphat Homestay,1830377
6,GMI lodge Kerema,2094144


In [40]:
BestHotels = pd.merge(hotel_df, perfect_df, on="CityID")

BestHotels.set_index("CityID", inplace=True)
BestHotels

,hotel_name,Unnamed: 0,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
CityID,,,,,,,,,,,,
5848280,Courtyard by Marriott Kaua'i at Coconut Beach,40,Kapaa,78,73.18,22.08,1,5.82,75.20,-159.32,"{'lon': -159.32, 'lat': 22.08}",US
2137773,OURE TERA beach resort,68,Vao,73,77.68,-22.67,0,15.46,77.68,167.48,"{'lon': 167.48, 'lat': -22.67}",NC
3836277,Colon Hotel de Campo Resort & Spa,261,Santa Fe,85,74.48,-31.63,0,8.01,75.99,-60.70,"{'lon': -60.7, 'lat': -31.63}",AR
3617154,Hotel Liwa Mairin,402,Puerto Cabezas,88,72.46,14.04,1,6.96,72.46,-83.39,"{'lon': -83.39, 'lat': 14.04}",NI
3671450,Fuente del Guania Hotel de Lujo,463,Inirida,87,76.19,3.87,1,1.68,76.19,-67.92,"{'lon': -67.92, 'lat': 3.87}",CO


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{CountryCode}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in BestHotels.iterrows()]
locations = BestHotels[["Latitude", "Longitude"]]


In [42]:
# Add marker layer ontop of heat map


# Display Map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))